# qualprep library: Example

This notebook provides an introduction to the qualprep library. It illustrates how the library can be used to prepare data, particularly survey data retrieved from Qualtrics. We start by providing an overview of the key function and then walk through a hypothetical example. 

# Function overview

The key function of the qualprep library is ***create_data***. This function creates a normalized and aggregated dataset with multi-category variables split. The following list provides an overview of the parameters. We will discuss each parameter in detail in our example. <br>

|**Parameter** | Description |
|:----------|:--------------------------------------------------------------------------------------------------|
|***data***  | (pd df) Raw data as pd df. Index will be disregarded should not be meaningful. |     
|***split_info***| (pd df) Instructions on the variables that need to be splitted. Some variables come as lists or strings. These need to be splitted into single variables. Example: 5,6. The split_info is a pd df with the two columns *variable_name* and *values_and_labels*. variable_name contains the names of the variables to be splitted. *values_and_labels* contains details on possible values and their labels. *values_and_labels* needs to be a dictionary.  Example: activities --> {'5': 'eating', '6': 'sleeping', '7': 'flying'}|
|***normalization_info***| (pd df) File containing instructions for the normalization of a variable. *normalization_info* is a pandas data frame with the first column containing the rawstring and the following columns all applicable normalizations. Example 1: All tokens of prickly pear (e.g., prickly pear, pricklypear, opuntia) should be normalized to "prickly pear." Example 2: "Vermillion and Ghila" is replaces with two entries named "Vermillion fly catcher" and "Ghila woodpecker."|
|***normalization_variable***| (str) The variable that needs to be normalized. |
|***aggregation_information***| (pd df) Instructions for aggregation. This is a pandas dataframe with the two columns *variable* and *agg_function*. *variable* contains the variables that need to be aggregated. *agg_function* specified the function to be applied for aggreation. Currently implemented are mean, median, max, min, dummy, and one to six. One to six aggregate into 1 if the respective number is present and into 0 otherwise. Dummy creates dummy variables first and then aggregates using the "max" function.*Note: This file needs to be in line with the split_info file. Make sure to include the newly created variables as defined in split_info in the aggregation_information to include them in the final dataset.* |
|***aggregation_variable***| (str) The variable along which we want to aggregate. This is the variable as it is named in the data. This is often the same as *normalization_variable*. |
|***aggregation_category_dict***| (dict) Used if higher-level aggregation on a categorical variable is desired. This is often applied to the variable that is normalized. For example, if we have animals and plants in our variable and we would like to combine all the animals into "animals" and all the plants into "plants" like "horse", "bird", "dog" -> animals and "bush", "tree", "cactus" -> plants. We would add a dict here that shows for every token if it is an animal or a plant. |
|***aggregation_category_variable***| (str) The variable to which the category aggregation is to be applied.|

The function returns an aggregated dataset containing the variables specified in aggregation_information and/or split_information. The variables are aggregated according to the instruction in the aggregation_information file. The function can be used to perform all the tasks (i.e., normalization, splitting, aggregation) or a selection of tasks only. 

# create_data: walk-trough

We first load the libraries, data, and instruction files. 

In [2]:
# Load library
import qualprep.qualprepfunctions as qp # qualprep for data prepration
import pandas as pd # padas for data handling

In [3]:
# Load data
data = pd.read_excel("data.xlsx")
normalization = pd.read_excel("normalization.xlsx")
split_info = pd.read_excel("instructions.xlsx", sheet_name = 'split')
aggregation_information = pd.read_excel("instructions.xlsx", sheet_name = 'aggregation')

In [4]:
# We can also get function documentation using "help"
help(qp.create_data)

Help on function create_data in module qualprep.qualprepfunctions:

create_data(data, split_info=None, normalization_info=None, normalization_variable=None, aggregation_category_dict=None, aggregation_category_variable=None, aggregation_information=None, aggregation_variable=None)
    Function to create an aggregated dataset containing specified variables. 
    
    Parameters
    ----------
        
        data: pd df
            Raw data. Index will be disregarded should not be meaningful. 
        
        split_info: pd df with nested dict
            Instructions on the variables that need to be splitted. Some variables come as lists or
            strings. These need to be splitted into single variables. Example: 5,6. The split_info is a pd df
            with the two columns variable_name and values_and_labels. variable_name contains the names of the
            variables to be splitted. values_and_labels contains details on possible values and their labels.
            values_

## Example dataset: data challenges

Assume, we want to analyze the behavior and habitat of different bird species. We received data of 12 sights of birds. The data looks as follows: 

In [5]:
data.head(12)

,sight_num,species,activity,sight_duration,habitat
0,1,Cardinal,"1, 2",21,woodland
1,2,Cardenal,"2,3",8,woodland
2,3,Trochilus Lucifer and Archilochus Alexandri,5,19,scrub
3,4,Ghila woodpecker,1,17,desert
4,5,Eckhornwoodpecker,2,90,woodland
5,6,Eckhorn woodpecker,2,10,woodland
6,7,Ghila and eckhorn woodpeckers,"1,3",17,desert
7,8,quail,4,5,scrub
8,9,archilochus alexandri,"5,1,3",12,woodland
9,10,Lucifer hummingbird fighting with broad-billde...,3,5,scrub


This data set has several problems: 
1. The species variable contains different representation of the same bird species. This includes misspellings (e.g., Cardenal instead of Cardinal), use of Latin and English names (e.g., archilochus alexandri and black-chinned hummingbird), and additional descpritives (e.g., female quail). 
2. Some sights include multiple species (e.g., Trochilus Lucifer and Archilochus Alexandri).
3. Activities are reported as numbers in a list-like manner and with irregular spacing (e.g., 1, 2 and 5,1,3). 
4. The dataset includes categorical and numerical variables which complicates aggregation. 

In the following, we show how qualprep's *create_data* function can help us address these problems. 

## Normalize string variable

First, we want to normalize the species names. For example, all representation of cardinals (i.e., Cardinal, Cardenal, and young cardinal) should be changed to "cardinal." For this purpose, we constructed a normalization file. The first row of the file contains the rawstring (i.e., the species name as it is in our origingal dataset). The following column(s) show the normalized version(s) associated with the raw string. If only one bird species was observed, the normalization file contains only one normalized name (norm_1; norm_2 bing NaN). If two species were reported (e.g., Trochilus Lucifer and Archilochus Alexandri), it contains two normalizations (norm_1 and norm_2). In our example, the normalization file stops at norm_2 because the maximum number of species reported in a sight is two. However, qualprep is designed to handle more columns. 

In [6]:
# Normalization file as pd df
normalization.head(7)

,rawstring,norm_1,norm_2
0,Cardinal,cardinal,NaN
1,Cardenal,cardinal,NaN
2,young cardinal,cardinal,NaN
3,Eckhornwoodpecker,eckhornwoodpecker,NaN
4,Eckhorn woodpecker,eckhornwoodpecker,NaN
5,Ghila woodpecker,ghila woodpecker,NaN
6,Ghila and eckhorn woodpeckers,ghila woodpecker,eckhornwoodpecker


We use qualprep's create_data function and the normalization file to normalize the data. 

In [7]:
# Normalize species variable
data_normalized = qp.create_data(data, normalization_info = normalization, normalization_variable = "species")
data_normalized.head(10)

100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 768.06it/s]


,sight_num,species,activity,sight_duration,habitat
0,1,cardinal,"1, 2",21,woodland
1,2,cardinal,"2,3",8,woodland
2,3,lucifer hummingbird,5,19,scrub
3,3,black-chinned hummingbird,5,19,scrub
4,4,ghila woodpecker,1,17,desert
5,5,eckhornwoodpecker,2,90,woodland
6,6,eckhornwoodpecker,2,10,woodland
7,7,ghila woodpecker,"1,3",17,desert
8,7,eckhornwoodpecker,"1,3",17,desert
9,8,quail,4,5,scrub


In the resulting dataset, the species variable is normalized as specified in the normalization file. Rows reporting multiple sights were multiplied with one row for each sight (e.g., sight_num 7). 

*Note: The create_data function ignores capitalization in the rawstring in the normalization file. All rawstrings are converted to lower case before use.* 

## Split categorical variable

Next, we want to split the activity variable into meaningful sub-variables. Each number in this variable represents an activity. For example, "1" means that the bird was eating, "2" it was hobbing around, etc. Our goal is to create new variables for each activity that shows whether or not the bird observed did the respective activity. 
To do this, we use the split_info file. The first column of the file specifies the variable name. The second column contains the dictionary with values and labels. The dictionary shows the numeric value as key and the respective activity (more precisely, activity variable to be created) as value. In our illustrative example, the *split_info* file contains only one variable. In more compley data sets with multiple variables to be split, the *split_info* file includes all variables that need splitting. 

In [8]:
# Split info 
split_info.head()

,variable_name,values_and_labels
0,activity,"{'1': 'activity_eating', '2': 'activity_hobbin..."


In [9]:
# Use qualprep to split the activities variable (as defined in the split_info file)
data_normalized_split = qp.create_data(data, normalization_info = normalization, normalization_variable = "species", 
               split_info = split_info)
data_normalized_split.head(5)

100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 768.80it/s]


,sight_num,species,sight_duration,habitat,activity_eating,activity_hobbing_around,activity_singing,activity_scratching_ground,activity_fighting
0,1,cardinal,21,woodland,1,0,0,0,0
1,2,cardinal,8,woodland,0,1,1,0,0
2,3,lucifer hummingbird,19,scrub,0,0,0,0,1
3,3,black-chinned hummingbird,19,scrub,0,0,0,0,1
4,4,ghila woodpecker,17,desert,1,0,0,0,0


Appling the *create_data* function and the *split_info* file, we can create a dataset in which the bird species variable is normalized and the activity variable split into separate variables. It is also possible to just split variables (without normalization) by leaving the normalization commands out (e.g., *qp.create_data(data, split_info = split_info)*). 

## Aggregate data

Next, we would like to aggregate the data by bird species. We want to use different aggregation methods for the different variables, namely the mean for sight duration, dummy variables for the habitat variable, and the maximum for the newly created activities variables. The aggregation_information file specifies the variable and the aggregation method to be applied. 

In [10]:
# Aggregation information
aggregation_information.head()

,variable,agg_function
0,sight_duration,mean
1,habitat,dummy
2,activity_eating,max
3,activity_hobbing_around,max
4,activity_singing,max


*Note: Make sure to include variables newly created through splitting using the variable names defined in the dictionary in the split_info file.*

We now use this file in combination with the *normalization_info* and *split_info* files to create an aggregated dataset. The parameter *aggregation_variable* specifies the variable along which we want to aggregated the dataset. In this case, this is the bird species variable.  

In [11]:
# Aggregate file as specified in aggregation_information
data_normalized_split_aggregated = qp.create_data(data, normalization_info = normalization, 
                                                  normalization_variable = "species", 
                                                  split_info = split_info, 
                                                  aggregation_information = aggregation_information, 
                                                  aggregation_variable = "species")
data_normalized_split_aggregated.head()

100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 191.94it/s]


,species,sight_duration,activity_eating,activity_hobbing_around,activity_singing,activity_scratching_ground,activity_fighting,habitat_desert,habitat_scrub,habitat_woodland
0,cardinal,17.333333,1,1,1,0,0,0,0,1
1,lucifer hummingbird,12.000000,0,0,1,0,1,0,1,0
2,black-chinned hummingbird,15.500000,1,0,1,0,1,0,1,1
3,ghila woodpecker,17.000000,1,0,1,0,0,1,0,0
4,eckhornwoodpecker,39.000000,1,1,1,0,0,1,0,1


The resulting data is aggregated by the normalized bird species according to our instructions. Instead of aggregating on species, we could also aggregate on habitat. 

In [12]:
# Normalize the bird species and aggregate on the habitat variable
# Change the aggregation_information
aggregation_information.iloc[1,0] = "species" # create dummy variables for the bird species

# Aggregate file as specified in aggregation_information
data_normalized_split_aggregated_v2 = qp.create_data(data, normalization_info = normalization, 
                                                  normalization_variable = "species", 
                                                  split_info = split_info, 
                                                  aggregation_information = aggregation_information, 
                                                  aggregation_variable = "habitat")
data_normalized_split_aggregated_v2.head()

100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 208.04it/s]


,habitat,sight_duration,activity_eating,activity_hobbing_around,activity_singing,activity_scratching_ground,activity_fighting,species_black-chinned hummingbird,species_broad-billed hummingbird,species_cardinal,species_eckhornwoodpecker,species_ghila woodpecker,species_lucifer hummingbird,species_quail
0,woodland,27.333333,1,1,1,0,1,1,0,1,1,0,0,0
1,scrub,10.000000,0,0,1,1,1,1,1,0,0,0,1,1
2,desert,17.000000,1,0,1,0,0,0,0,0,1,1,0,0


## Higher level aggregation

Finally, we may be interested in overarching species (e.g., woodpecker or hummingbird). Hence, we want to aggregate the data on these overarching species. The *species_type_dict* defined below shows for every species type (normalized) the overarching species. We can use this dictionary to do the respective aggregation. 

In [13]:
# Dicionnary of overarching species
species_type_dict = {"cardinal": "other", 
                     "ghila woodpecker": "woodpecker", 
                     "eckhornwoodpecker": "woodpecker", 
                     "quail": "other", 
                     "black-chinned hummingbird": "hummingbird", 
                     "lucifer hummingbird": "hummingbird", 
                     "broad-billed hummingbird": "hummingbird"}

In [14]:
# Aggregate file as specified in aggregation_information and species_type_dict
data_normalized_split_aggregated_v3 = qp.create_data(data, normalization_info = normalization, 
                                                     normalization_variable = "species", 
                                                     split_info = split_info, 
                                                     aggregation_category_dict = species_type_dict, 
                                                     aggregation_category_variable = "species",
                                                     aggregation_information = aggregation_information, 
                                                     aggregation_variable = "species")
data_normalized_split_aggregated_v3

100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 191.87it/s]


,species,sight_duration,activity_eating,activity_hobbing_around,activity_singing,activity_scratching_ground,activity_fighting,species_hummingbird,species_other,species_woodpecker
0,other,12.8,1,1,1,1,0,0,1,0
1,hummingbird,12.0,1,0,1,0,1,1,0,0
2,woodpecker,30.2,1,1,1,0,0,0,0,1


The resulting dataset shows for the three species types *woodpecker*, *hummingbirs*, and *other* the average sight duration, presence or absence of each activity, and the habitats in which the type was observed. 